Run next cell only once


In [1]:
%pip install newspaper3k
%pip install sumy

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))) - skipping
Note: you may need to restart the kernel to use updated packages.

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))) - skipping


Start running here

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AK04726P\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import newspaper
from requests.exceptions import Timeout
from textblob import TextBlob

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


# Download the VADER lexicon
nltk.download('vader_lexicon')

def summarize_text(text, max_sentences=3):
    # Create a parser using PlaintextParser
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # Create an LSA summarizer
    summarizer = LsaSummarizer()
    # Summarize the text
    summary = summarizer(parser.document, max_sentences)
    # Return the summary as a string
    return ' '.join(str(sentence) for sentence in summary)

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores['compound']

def web_scrape_clients(clients, max_number_news=5):
    news_dfs = []

    for client in clients:
        # Remove commas and replace spaces with plus signs
        client = client.replace(",", "").replace(" ", "+")

        url = f"https://www.bing.com/news/search?q={client}+-msn"
        response = requests.get(url, verify=False)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            headers = [a.text for a in soup.select(".t_t a.title")]
            journals = [a["data-author"] for a in soup.select(".t_t a.title")]
            links = [a["href"] for a in soup.select(".t_t a.title")]
            snippet = [s.text for s in soup.select(".snippet")]
            summary = []
            sentiment = []

            num_news = min(len(headers), max_number_news)

        summary.append("error")
        sentiment.append(0)  # Assuming default sentiment value for error cases

        # Update the code inside the loop where summaries and sentiments are appended
        for link in links:

            try:
                # Use requests to download the page content with a timeout of 5 seconds
                response = requests.get(link, timeout=20)
                page_content = response.content

                article = newspaper.Article(link)
                article.download(input_html=page_content)
                article.parse()
                article.nlp()

                if not article.text:
                    sentiment.append("-")
                    summary.append("Could not get the summary for this article: Subscription needed.")
                    continue  # Skip articles with empty text

                sentiment.append(get_sentiment(article.text))
                summary.append(summarize_text(article.text))

                if len(summary) == max_number_news:
                    break  # Stop when desired number of non-empty summaries is reached

            except Timeout:
                summary.append("error")
                sentiment.append(0)  # Default sentiment for error case
            except Exception as e:
                summary.append("error")
                sentiment.append(0)  # Default sentiment for error case



            news_df = pd.DataFrame({
                "Client": [client] * num_news,
                "Header": headers[:num_news],
                "Snippet": snippet[:num_news],
                "Journal": journals[:num_news],
                "Link": links[:num_news],
                "Summary" : summary[:num_news],
                "Sentiment": sentiment[:num_news]
            })

            news_dfs.append(news_df)

    final_news_df = pd.concat(news_dfs, ignore_index=True)
    final_news_df["Client"] = final_news_df["Client"].str.replace("+", " ")
    final_news_df["Link"] = final_news_df["Link"].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')



    return final_news_df


IndentationError: unexpected indent (3880915978.py, line 94)

After running the following line, it will ask you to type a client name and press enter. Once the table loads, you can click on the little table icon on the right to read the summaries better

In [14]:
def get_user_input():
    clients = input("Enter a list of clients separated by commas: ")
    clients = clients.split(",")
    clients = [client.strip() for client in clients]
    return clients

def main():
    clients = get_user_input()
    final_news_df = web_scrape_clients(clients, max_number_news=5)

    # Display the DataFrame
    from IPython.display import display, HTML
    # Display the DataFrame with clickable hyperlinks
    display(HTML(final_news_df.to_html(escape=False)))
    #display(final_news_df)
    #final_news_df.to_csv('news_data.csv', index=False)  # Save as CSV

if __name__ == "__main__":
    main()


c:\Users\AK04726P\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ValueError: All arrays must be of the same length